# Product Funnel Optimization — Interactive Dashboard (Dash)
Purpose: Interactive exploration of funnel performance, trends, and conversion drivers.

In [53]:
import pandas as pd
import numpy as np

import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [54]:
# Load data
events = pd.read_csv("../data/events.csv", parse_dates=["event_timestamp"])
users = pd.read_csv("../data/users.csv", parse_dates=["signup_date"])
products = pd.read_csv("../data/products.csv")

# User funnel (validated earlier)
user_funnel = (
    events
    .assign(
        viewed=lambda x: (x["event_type"] == "view").astype(int),
        added_to_cart=lambda x: (x["event_type"] == "cart").astype(int),
        wishlisted=lambda x: (x["event_type"] == "wishlist").astype(int),
        purchased=lambda x: (x["event_type"] == "purchase").astype(int),
    )
    .groupby("user_id")[["viewed", "added_to_cart", "wishlisted", "purchased"]]
    .max()
    .reset_index()
)

# Funnel counts
funnel_counts = pd.DataFrame({
    "stage": ["View", "Cart", "Wishlist", "Purchase"],
    "users": [
        user_funnel["viewed"].sum(),
        user_funnel["added_to_cart"].sum(),
        user_funnel["wishlisted"].sum(),
        user_funnel["purchased"].sum()
    ]
})

# Weekly conversion
weekly = (
    events
    .assign(week=lambda x: x["event_timestamp"].dt.to_period("W").dt.start_time)
    .groupby("week")
    .agg(
        viewers=("event_type", lambda x: (x=="view").sum()),
        buyers=("event_type", lambda x: (x=="purchase").sum())
    )
    .reset_index()
)

weekly["conversion_rate"] = weekly["buyers"] / weekly["viewers"] * 100

# Category conversion
category_conv = (
    events
    .merge(products, on="product_id")
    .groupby("category")
    .agg(
        viewers=("event_type", lambda x: (x=="view").sum()),
        buyers=("event_type", lambda x: (x=="purchase").sum())
    )
    .reset_index()
)

category_conv["conversion_rate"] = (
    category_conv["buyers"] / category_conv["viewers"] * 100
)

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Build modeling table (same logic as before)
user_features = (
    events
    .groupby("user_id")
    .agg(
        num_views=("event_type", lambda x: (x == "view").sum()),
        num_cart_adds=("event_type", lambda x: (x == "cart").sum()),
        num_wishlists=("event_type", lambda x: (x == "wishlist").sum()),
        num_purchases=("event_type", lambda x: (x == "purchase").sum()),
    )
    .reset_index()
)

user_features["converted"] = (user_features["num_purchases"] > 0).astype(int)

# Merge signup date for tenure
user_features = user_features.merge(
    users[["user_id", "signup_date"]],
    on="user_id",
    how="left"
)

max_event_date = events["event_timestamp"].max()
user_features["tenure_days"] = (
    max_event_date - user_features["signup_date"]
).dt.days

# Features & target
X = user_features[["num_views", "num_cart_adds", "num_wishlists", "tenure_days"]]
y = user_features["converted"]

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit model
model = LogisticRegression(max_iter=1000)
model.fit(X_scaled, y)
user_features["conversion_probability"] = model.predict_proba(X_scaled)[:, 1]

# Create coef_df (THIS was missing)
coef_df = (
    pd.DataFrame({
        "feature": X.columns,
        "coefficient": model.coef_[0]
    })
    .sort_values("coefficient")
)

In [68]:
import plotly.graph_objects as go

# ===== New Executive Slate Theme =====
BG = "#0b0f14"          # deep slate (page background)
CARD_BG = "#121821"     # elevated card surface
ACCENT = "#2dd4bf"      # muted teal (primary accent)
ACCENT_SOFT = "#5eead4" # lighter teal (secondary highlight)

TEXT = "#e5e7eb"        # soft off-white
MUTED = "#9ca3af"       # muted gray text
GRID = "#1f2937"        # subtle gridlines

def style_fig(fig, title=None):
    fig.update_layout(
        title=title,
        paper_bgcolor=BG,
        plot_bgcolor=BG,
        font=dict(color=TEXT, size=13),
        title_font=dict(size=16, color=TEXT),
        xaxis=dict(
            showgrid=True,
            gridcolor=GRID,
            zeroline=False,
            showline=False
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor=GRID,
            zeroline=False,
            showline=False
        ),
        margin=dict(l=32, r=24, t=48, b=32)
    )
    return fig

In [85]:
from dash import html

def kpi_card(title, value, subtitle="", highlight=False):
    return html.Div(
        children=[
            html.Div(title, style={"fontSize":12, "color":MUTED}),
            html.Div(
                value,
                style={
                    "fontSize":26,
                    "fontWeight":"700",
                    "color": "#4BB543" if highlight else TEXT
                }
            ),
            html.Div(subtitle, style={"fontSize":11, "color":MUTED})
        ],
        style={
            "flex":"1",
            "padding":"18px",
            "background":CARD_BG,
            "borderRadius":"16px",
            "margin":"6px",
            "minWidth":"180px"
        }
    )

In [58]:
funnel_pct = pd.DataFrame({
    "stage": ["View → Cart", "Cart → Purchase"],
    "drop_rate": [
        (funnel_counts.loc[0,"users"] - funnel_counts.loc[1,"users"]) / funnel_counts.loc[0,"users"] * 100,
        (funnel_counts.loc[1,"users"] - funnel_counts.loc[3,"users"]) / funnel_counts.loc[1,"users"] * 100
    ]
})

In [59]:
fig_drop_pct = px.bar(
    funnel_pct,
    x="stage",
    y="drop_rate",
    text=funnel_pct["drop_rate"].round(1),
    labels={"drop_rate":"Drop-off (%)"}
)
fig_drop_pct.update_traces(marker_color=WARNING, textposition="outside")
fig_drop_pct = style_fig(fig_drop_pct, "Funnel Drop-Off Rates")

In [60]:
fig_prob_dist = px.histogram(
    user_features,
    x="conversion_probability",
    nbins=40
)
fig_prob_dist.update_traces(marker_color=ACCENT)
fig_prob_dist = style_fig(fig_prob_dist, "Distribution of Conversion Probability")

In [61]:
scatter_df = user_features.copy()
scatter_df["converted_label"] = scatter_df["converted"].map({1:"Converted",0:"Not Converted"})

fig_scatter = px.scatter(
    scatter_df,
    x="num_views",
    y="num_cart_adds",
    color="converted_label",
    opacity=0.4
)
fig_scatter = style_fig(fig_scatter, "Views vs Cart Adds (Conversion Behavior)")

In [62]:
fig_funnel = px.bar(
    funnel_counts,
    x="stage",
    y="users",
    title="User Funnel Overview"
)
fig_funnel.update_traces(marker_color=ACCENT)
fig_funnel = apply_dark(fig_funnel)

In [63]:
fig_trend = px.line(
    weekly,
    x="week",
    y="conversion_rate",
    title="Weekly View → Purchase Conversion Rate",
    labels={"conversion_rate":"Conversion Rate (%)"}
)
fig_trend.update_traces(line=dict(width=3, color=ACCENT))
fig_trend = apply_dark(fig_trend)

In [64]:
fig_category = px.bar(
    category_conv.sort_values("conversion_rate", ascending=False),
    x="category",
    y="conversion_rate",
    title="Conversion Rate by Product Category"
)
fig_category.update_traces(marker_color=ACCENT)
fig_category = apply_dark(fig_category)

In [65]:
fig_model = px.bar(
    coef_df,
    x="coefficient",
    y="feature",
    orientation="h",
    title="Behavioral Drivers of Conversion (Logistic Regression)"
)
fig_model.update_traces(marker_color=ACCENT)
fig_model = apply_dark(fig_model)

In [86]:
import dash
from dash import dcc, html

app = dash.Dash(__name__)
app.title = "Product Funnel Optimization"

# -------------------------------------------------
# Kill browser / iframe white border (GLOBAL FIX)
# -------------------------------------------------
app.index_string = """
<!DOCTYPE html>
<html>
  <head>
    {%metas%}
    <title>{%title%}</title>
    {%favicon%}
    {%css%}
    <style>
      html, body {
        margin: 0;
        padding: 0;
        background-color: #0b0f14;
      }
    </style>
  </head>
  <body>
    {%app_entry%}
    <footer>
      {%config%}
      {%scripts%}
      {%renderer%}
    </footer>
  </body>
</html>
"""

# -------------------------------------------------
# Tab Styles (MUST be defined outside layout)
# -------------------------------------------------
TAB_STYLE = {
    "backgroundColor": BG,
    "color": ACCENT,
    "border": "none",
    "fontWeight": "500",
    "padding": "10px 16px",
    "fontSize": "14px"
}

TAB_SELECTED_STYLE = {
    "backgroundColor": CARD_BG,
    "color": ACCENT,
    "border": "none",
    "fontWeight": "700",
    "padding": "10px 16px",
    "fontSize": "14px"
}

# -------------------------------------------------
# Layout
# -------------------------------------------------
app.layout = html.Div(
    style={
        "backgroundColor": BG,
        "minHeight": "100vh",
        "width": "100vw",
        "margin": "0",
        "padding": "12px 16px",
        "fontFamily": "Inter, system-ui, -apple-system"
    },
    children=[

        # -------------------------
        # Header
        # -------------------------
        html.Div([
            html.H1(
                "Product Funnel Optimization",
                style={
                    "textAlign": "center",
                    "color": ACCENT,
                    "marginBottom": "4px",
                    "fontWeight": "700"
                }
            ),
            html.Div(
                "Interactive analysis of conversion behavior and funnel efficiency",
                style={
                    "textAlign": "center",
                    "color": MUTED,
                    "fontSize": "14px"
                }
            )
        ], style={"marginBottom": "24px"}),

        # -------------------------
        # KPI Row
        # -------------------------
        html.Div([
            kpi_card("Total Users", f"{user_funnel.shape[0]:,}"),
            kpi_card(
                "View → Purchase Rate",
                f"{weekly['conversion_rate'].mean():.2f}%",
                highlight=True
            ),
            kpi_card("Largest Drop-Off", "Cart → Purchase"),
            kpi_card(
                "Avg Weekly Conversion",
                f"{weekly['conversion_rate'].mean():.2f}%"
            )
        ], style={
            "display": "flex",
            "flexWrap": "wrap",
            "justifyContent": "space-between",
            "marginBottom": "28px"
        }),

        # -------------------------
        # Tabs (FINAL, BORDERLESS)
        # -------------------------
        dcc.Tabs(
            value="overview",
            style={
                "backgroundColor": BG,
                "border": "none"
            },
            colors={
                "border": "none",
                "background": BG,
                "primary": ACCENT
            },
            children=[

                # ===== Overview =====
                dcc.Tab(
                    label="Overview",
                    value="overview",
                    style=TAB_STYLE,
                    selected_style=TAB_SELECTED_STYLE,
                    children=[
                        html.Div([
                            html.Div(
                                dcc.Graph(figure=fig_funnel),
                                style={"flex": "2", "margin": "6px"}
                            ),
                            html.Div(
                                dcc.Graph(figure=fig_drop_pct),
                                style={"flex": "1", "margin": "6px"}
                            )
                        ], style={"display": "flex", "flexWrap": "wrap"})
                    ]
                ),

                # ===== Trends =====
                dcc.Tab(
                    label="Trends",
                    value="trends",
                    style=TAB_STYLE,
                    selected_style=TAB_SELECTED_STYLE,
                    children=[
                        html.Div(
                            dcc.Graph(figure=fig_trend),
                            style={"margin": "6px"}
                        )
                    ]
                ),

                # ===== Product =====
                dcc.Tab(
                    label="Product Performance",
                    value="product",
                    style=TAB_STYLE,
                    selected_style=TAB_SELECTED_STYLE,
                    children=[
                        html.Div(
                            dcc.Graph(figure=fig_category),
                            style={"margin": "6px"}
                        )
                    ]
                ),

                # ===== Behavior & ML =====
                dcc.Tab(
                    label="Behavior & ML",
                    value="behavior",
                    style=TAB_STYLE,
                    selected_style=TAB_SELECTED_STYLE,
                    children=[

                        html.Div([
                            html.Div(
                                dcc.Graph(figure=fig_model),
                                style={"flex": "1", "margin": "6px"}
                            ),
                            html.Div(
                                dcc.Graph(figure=fig_prob_dist),
                                style={"flex": "1", "margin": "6px"}
                            )
                        ], style={"display": "flex", "flexWrap": "wrap"}),

                        html.Div(
                            dcc.Graph(figure=fig_scatter),
                            style={"margin": "6px"}
                        )
                    ]
                )
            ]
        ),

        # -------------------------
        # Footer
        # -------------------------
        html.Div(
            "Dashboard built with Dash & Plotly · SQL-validated · Explainable ML",
            style={
                "textAlign": "center",
                "color": MUTED,
                "fontSize": "12px",
                "marginTop": "18px"
            }
        )
    ]
)

In [87]:
print("Dashboard running at http://127.0.0.1:8050")
app.run(debug=True, use_reloader=False, port=8050)

Dashboard running at http://127.0.0.1:8050
